# 06-Trajectory Querying

This notebooks exemplifies the querying of quadkey-indexed trajectories using the Extended Vehicle Energy Dataset.

**Requirements**: Run the `calculate-trajectories.py` script before running this notebook.

In [ ]:
import folium
import numpy as np
import pandas as pd
import osmnx as ox
import geopandas as gpd
import networkx as nx
import math

from itertools import pairwise
from db.api import EVedDb
from folium.vector_layers import PolyLine, CircleMarker
from pyquadkey2 import quadkey
from numba import jit
from db.api import EVedDb
from tqdm.notebook import tqdm
from raster.drawing import smooth_line
from geo.qk import tile_to_str

We start by loading the road network from Ann Arbor, Michigan, using the `OSMnx` package.

In [ ]:
def load_road_network(place_name, network_type='drive'):
    g = ox.graph_from_place(place_name, network_type=network_type, simplify=False)
    g = ox.add_edge_speeds(g)
    g = ox.add_edge_travel_times(g)
    g = ox.bearing.add_edge_bearings(g)
    return g

In [ ]:
g = load_road_network('Ann Arbor, Michigan')

The `geocode_address` function converts an address to a geospatial coordinate.

In [ ]:
def geocode_address(address, crs=4326):
    geocode = gpd.tools.geocode(address, 
                                provider='nominatim', 
                                user_agent="QuadKey trajectory query").to_crs(crs)
    return geocode.iloc[0].geometry.y, geocode.iloc[0].geometry.x

The `find_route` below uses the `geocode_address` function to generate a route between any two given addresses. The  generated route is a list of road network nodes.

In [ ]:
def find_route(g, addr_ini, addr_end, weight='travel_time'):
    loc_ini = geocode_address(addr_ini)
    loc_end = geocode_address(addr_end)
    node_ini = ox.distance.nearest_nodes(g, loc_ini[1], loc_ini[0])
    node_end = ox.distance.nearest_nodes(g, loc_end[1], loc_end[0])
    route = nx.shortest_path(g, node_ini, node_end, weight=weight)
    return route

In [ ]:
route = find_route(g,
                   addr_ini="122 N Thayer St, Ann Arbor, MI 48104, USA",
                   addr_end="1431 Ardmoor Ave, Ann Arbor, MI 48103, USA")

The `fit_bounding_box` uses a list of locations to fit a bounding box for the displayed data and set the appropriate map center and zoom.

In [ ]:
def fit_bounding_box(html_map, bb_list):
    if isinstance(bb_list, list):
        ll = np.array(bb_list)
    else:
        ll = bb_list
        
    min_lat, max_lat = ll[:, 0].min(), ll[:, 0].max()
    min_lon, max_lon = ll[:, 1].min(), ll[:, 1].max()
    html_map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    return html_map

In [ ]:
def map_route(g, route):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)
    
    empty_edges = []
    bb_list = []
    
    for n in route:
        loc = g.nodes[n]
        # CircleMarker((loc['y'], loc['x']), radius=2, color="red", fill="red", opacity=0.5, tooltip=n, popup=n).add_to(html_map)
        bb_list.append((loc['y'], loc['x']))
    
    for n0, n1 in pairwise(route):
        edge = g[n0][n1]
        l0 = g.nodes[n0]
        l1 = g.nodes[n1]
        line = [(l0['y'], l0['x']), (l1['y'], l1['x'])]
        
        PolyLine(line, weight=5, opacity=0.5, popup=edge[0]).add_to(html_map)
        
    return fit_bounding_box(html_map, bb_list)

In [ ]:
map_route(g, route)

The `get_qk_line` function _draws_ a quadkey line between two endpoints and returns it as a list.

In [ ]:
def get_qk_line(loc0, loc1, level):
    qk0 = quadkey.from_geo((loc0['y'], loc0['x']), level)
    qk1 = quadkey.from_geo((loc1['y'], loc1['x']), level)
    
    ((tx0, ty0), _) = qk0.to_tile()
    ((tx1, ty1), _) = qk1.to_tile()

    line = smooth_line(tx0, ty0, tx1, ty1)
    return [(quadkey.from_str(tile_to_str(int(p[0]), int(p[1]), level)), p[2]) for p in line if p[2] > 0.0]

The `get_route_quadkeys` generates the unique quadkey codes that correspond to the route gepgraphy.

In [ ]:
def get_route_quadkeys(g, route, level=20):
    qks = set()
    shift = 64 - 2 * level
    for n0, n1 in pairwise(route):
        edge = g[n0][n1]
        l0 = g.nodes[n0]
        l1 = g.nodes[n1]
        qks.update([(qk.to_quadint() >> shift, edge[0]['bearing']) \
                    for qk, _ in get_qk_line(l0, l1, level)])
    return list(qks)

The `load_signal_range` function loads a sequence of unique locations from a range of indices.

In [ ]:
def load_signal_range(r):
    db = EVedDb()
    sql = """
    select   match_latitude
    ,        match_longitude 
    from     signal 
    where    signal_id >= ? and signal_id <= ?
    """
    
    points = []
    all_points = db.query(sql, [int(r[0]), int(r[1])])
    for p0, p1 in pairwise(all_points):
        if len(points) == 0:
            points.append(p0)  
        elif p0 != p1:
            points.append(p1)
            
    return points

In [ ]:
def load_trajectory_quadkeys(traj_id):
    db = EVedDb()
    
    sql = """
    select     s.quadkey
    from       signal s
    inner join trajectory t on s.vehicle_id = t.vehicle_id and s.trip_id = t.trip_id
    where      t.traj_id = ?;
    """
    qks = {qk[0] for qk in db.query(sql, [traj_id])}
    return qks

In [ ]:
def load_trajectory_points(traj_id):
    db = EVedDb()
    
    sql = """
    select     distinct
               s.match_latitude
    ,          s.match_longitude
    from       signal s
    inner join trajectory t on s.vehicle_id = t.vehicle_id and s.trip_id = t.trip_id
    where      t.traj_id = ?
    order by   s.signal_id;
    """
    return db.query(sql, [traj_id])

In [ ]:
def load_link_points(link_id):
    db = EVedDb()
    
    get_range_sql = "select signal_ini, signal_end from link where link_id=?"
    ranges = db.query(get_range_sql, [link_id])
    
    if len(ranges):
        get_points_sql = """
        select distinct match_latitude
        ,               match_longitude 
        from            signal 
        where           signal_id >= ? and signal_id <= ?
        order by        signal_id
        """
        return db.query(get_points_sql, [ranges[0][0], ranges[0][1]])
    else:
        return []

In [ ]:
load_link_points(1)

In [ ]:
def get_overlapping_links(g, route, level=20, angle_delta=2.5):
    qks = get_route_quadkeys(g, route, level)
    cos_angle_delta = math.cos(math.radians(angle_delta))
    
    sql = """
    select     q.link_id
    ,          l.traj_id
    ,          l.signal_ini
    ,          l.signal_end
    from       link_qk q
    inner join link l on l.link_id = q.link_id
    where      q.quadkey = ? and l.bearing > 0 and cos(radians(l.bearing - ?)) >= ?;
    """
    db = EVedDb()
    links = set()
    for qk, bearing in qks:
        links.update(db.query(sql, [qk, bearing, cos_angle_delta]))
    return np.array(list(links))

In [ ]:
@jit(nopython=True)
def get_contiguous_ranges(signal_ini, signal_end):
    ranges = np.zeros((signal_ini.shape[0], 2))
    ini = signal_ini[0]
    end = signal_end[0]
    
    j = 0
    for i in range(1, signal_ini.shape[0]):
        end = signal_end[i-1]
        if signal_ini[i] != end:
            ranges[j, 0] = ini
            ranges[j, 1] = end
            j += 1
            ini = signal_ini[i]
            
    if j == 0:
        ranges[j, 0] = ini
        ranges[j, 1] = end
        j += 1

    return ranges[:j, :]

In [ ]:
def get_matching_trajectories(g, route, level=20, angle_delta=2.5):
    links = get_overlapping_links(g, route, level, angle_delta)
    trajectories = np.unique(links[:, 1])
    return trajectories, links

In [ ]:
def get_overlapping_signal_ranges(g, route, level=20, angle_delta=2.5):
    trajectories, links = get_matching_trajectories(g, route, level, angle_delta)
    
    ranges = []
    for t in trajectories:
        index = links[:, 1] == t
        
        signal_ini = links[index, 2]
        signal_end = links[index, 3]
        ranges.extend(get_contiguous_ranges(signal_ini, signal_end).tolist())
    return ranges

In [ ]:
def map_matching_links(g, route):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)
    
    empty_edges = []
    bb_list = []
    
    ranges = get_overlapping_signal_ranges(g, route)
    for r in tqdm(ranges):
        line = load_signal_range(r)
        if len(line):
            bb_list.extend(line)
            PolyLine(line, weight=3, color="red", opacity=0.5, popup=r).add_to(html_map)

    line = []
    for n in route:
        loc = g.nodes[n]
        p = (loc['y'], loc['x'])
        line.append(p)
        bb_list.append(p)
        
    PolyLine(line, weight=5, opacity=0.5).add_to(html_map)

    return fit_bounding_box(html_map, bb_list)

In [ ]:
# map_matching_links(g, route)

In [ ]:
def jaccard_similarity(set0, set1):
    return len(set0 & set1) / len(set0 | set1)

In [ ]:
def calculate_trajectory_matches(g, route, level=20):
    trajectories, links = get_matching_trajectories(g, route, level)
    
    route_qks = {qk[0] for qk in get_route_quadkeys(g, route, level)}
    data = []
    for trajectory in trajectories:
        traj_qks = load_trajectory_quadkeys(int(trajectory))
        similarity = jaccard_similarity(traj_qks, route_qks)
        data.append((trajectory, similarity))
    return data

In [ ]:
match_df = pd.DataFrame(data=calculate_trajectory_matches(g, route), columns=['traj_id', 'similarity'])

In [ ]:
match_df["percent_rank"] = match_df["similarity"].rank(pct=True)

In [ ]:
match_df.sort_values("percent_rank", ascending=False)

In [ ]:
match_df[match_df["percent_rank"] > 0.95].sort_values("percent_rank", ascending=False)

In [ ]:
def get_top_match_trajectories_r(g, route, top=0.05):
    match_df = pd.DataFrame(data=calculate_trajectory_matches(g, route), columns=['traj_id', 'similarity'])
    match_df["percent_rank"] = match_df["similarity"].rank(pct=True)
    
    filtered_df = match_df[match_df["percent_rank"] > (1.0 - top)]
    trajectories = filtered_df["traj_id"].values
    return trajectories

In [ ]:
def map_top_matching_trajectories_r(g, route, top=0.05):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)
    
    empty_edges = []
    bb_list = []
    
    trajectories = get_top_match_trajectories_r(g, route, top=0.05)
    for traj_id in trajectories:
        line = load_trajectory_points(int(traj_id))
        if len(line) > 0:
            bb_list.extend(line)
            PolyLine(line, weight=3, color="red", opacity=0.5).add_to(html_map)

    line = []
    for n in route:
        loc = g.nodes[n]
        p = (loc['y'], loc['x'])
        line.append(p)
        bb_list.append(p)
        
    PolyLine(line, weight=5, opacity=0.5).add_to(html_map)

    return fit_bounding_box(html_map, bb_list)

In [ ]:
map_top_matching_trajectories_r(g, route)

## 06.03 Querying Using a Trajectory

In this section we will perform the same query but using a known trajectory instead

In [ ]:
def load_matching_links(traj_id, angle_delta=2.5):
    db = EVedDb()
    
    sql = """
    select     q.link_id
    ,          q.quadkey
    ,          l.traj_id
    from       link_qk q
    inner join link l on l.link_id = q.link_id
    inner join (

        select     q.quadkey
        ,          l.bearing
        from       link_qk q
        inner join link l on l.link_id = q.link_id
        where      l.traj_id = ?
    ) x on x.quadkey = q.quadkey
    where l.bearing > 0 and x.bearing > 0 and cos(radians(x.bearing - l.bearing)) >= cos(radians(?));
    """
    traj_df = db.query_df(sql, [traj_id, angle_delta])
    return traj_df

In [ ]:
def get_top_match_trajectories_t(traj_id, top=0.05):
    df = load_matching_links(traj_id)
    trajectories = np.unique(df["traj_id"].values)
    query_set = set(df[df["traj_id"] == traj_id]["quadkey"].values)
    
    traj_df = pd.DataFrame(data=trajectories, columns=["traj_id"])
    traj_df["similarity"] = [jaccard_similarity(query_set, set(df[df["traj_id"] == t]["quadkey"].values)) \
                             for t in trajectories]
    traj_df["percent_rank"] = traj_df["similarity"].rank(pct=True)
    
    filtered_df = traj_df[traj_df["percent_rank"] > (1.0 - top)]
    trajectories = filtered_df["traj_id"].values
    return trajectories    

In [ ]:
def get_matching_links_t(traj_id):
    df = load_matching_links(traj_id)
    links = np.unique(df["link_id"].values)
    return links    

In [ ]:
def map_top_matching_trajectories_t(traj_id, top=0.05):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)
    
    bb_list = []
    
    trajectories = get_top_match_trajectories_t(traj_id, top)
    for trajectory in trajectories:
        if trajectory != traj_id:
            line = load_trajectory_points(int(trajectory))
            if len(line) > 0:
                bb_list.extend(line)
                PolyLine(line, weight=3, color="red", opacity=0.5, popup=str(trajectory)).add_to(html_map)

    line = load_trajectory_points(int(traj_id))
    PolyLine(line, weight=5, opacity=0.5).add_to(html_map)

    return fit_bounding_box(html_map, bb_list)

In [ ]:
map_top_matching_trajectories_t(traj_id=4, top=0.01)

In [ ]:
def map_matching_links_t(traj_id):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron", max_zoom=20, control_scale=True)
    
    bb_list = []

    line = load_trajectory_points(int(traj_id))
    PolyLine(line, weight=12, opacity=0.5).add_to(html_map)
    
    bb_list.extend(line)
    
    links = get_matching_links_t(traj_id)
    print(len(links))
    for link in links:
        line = load_link_points(int(link))
        if len(line) > 0:
            bb_list.extend(line)
            PolyLine(line, weight=3, color="red", opacity=0.5, popup=str(link)).add_to(html_map)

    return fit_bounding_box(html_map, bb_list)

In [ ]:
map_matching_links_t(traj_id=4)